In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.types import StructType, IntegerType, StructField

In [3]:
from numpy import argmax

In [4]:
spark = SparkSession.builder.getOrCreate()

21/12/10 00:46:46 WARN Utils: Your hostname, Caballero resolves to a loopback address: 127.0.1.1; using 10.0.0.15 instead (on interface wlp0s20f3)
21/12/10 00:46:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/10 00:46:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 00:46:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/10 00:46:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
L = []
for d in range(10):
    df = spark.read.option("inferScheme", True).option("header",True).csv('component'+str(d)+'.csv')
    L.append([row.asDict()['Probability'] for row in df.collect()] ) 
del(df)
S = []
for j in range(len(L[0])):
    prob = [ float(L[d][j]) for d in range(10) ]

    S.append( (int(j+1), int(argmax(prob)) ) )
output_df = spark.createDataFrame( spark.sparkContext.parallelize(S), StructType([
    StructField("ImageId", IntegerType(), True),
    StructField("Label", IntegerType(), True)
]))
output_df.toPandas().to_csv('submission.csv', index = False)